In [1]:
import glob
import re
import random
import numpy as np
import pandas as pd

In [2]:
from src.extract import *
from src.preprocess import *

WORD_LEN = 12
PADDING_CHAR = '^' # Use ^ since it's rarely used in normal word

In [3]:
normal_words = get_words_from_files('data\\train\\WordPress-master\\*\\*.php')
normal_words = normal_words.union(get_words_from_files('data\\train\\ccxt-master\\*\\*.php'))
prep_normal_words = preprocess_words(normal_words, word_len=WORD_LEN, padding_char=PADDING_CHAR)
print(len(prep_normal_words), prep_normal_words[:10])

54520 ['^^^^^^^^^^^^', ' transfer ()', '40507^^^^^^^', '&#x1f486;&#x', 'richEditingE', 'Failed to pa', '3e2414d82acc', '&#x1f468;&#x', 'ADAUSDTM^^^^', '&#x1f1ec;&#x']


In [4]:
passwords = open('data/train/000webhost.txt', 'rb').read().decode().split('\n')
user_pass = passwords[:40000]
prep_user_pass = preprocess_words(user_pass, word_len=WORD_LEN, padding_char=PADDING_CHAR)
print(len(prep_user_pass), prep_user_pass[:10])

39984 ['1qaz2wsx^^^^', '306187mn^^^^', 'rados1^^^^^^', 'newyork911^^', 'abc123^^^^^^', 'taqiyudin100', 'wjr5443^^^^^', 'nana0428^^^^', '1992jp^^^^^^', 'bahamut24rit']


In [5]:
data = [
    [encode_str(word), 0]
    for word in prep_normal_words
] + [
    [encode_str(word), 1]
    for word in prep_user_pass
]

random.shuffle(data)
X = np.asarray([row[0] for row in data])
y = np.asarray([row[1] for row in data])

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
import tensorflow.keras as keras

model = keras.models.Sequential([
    keras.layers.SimpleRNN(30, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(10),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.Recall()]
    )

In [8]:
import os
import datetime

from tensorflow.keras.callbacks import EarlyStopping

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=256,
    validation_split=0.2,
    callbacks=[early_stopping, tensorboard_callback])

Epoch 1/100
222/222 [==============================] - 8s 19ms/step - loss: 0.5004 - recall: 0.5586 - val_loss: 0.3771 - val_recall: 0.7217
Epoch 2/100
222/222 [==============================] - 2s 9ms/step - loss: 0.3457 - recall: 0.7803 - val_loss: 0.3100 - val_recall: 0.7879
Epoch 3/100
222/222 [==============================] - 2s 9ms/step - loss: 0.2998 - recall: 0.8253 - val_loss: 0.2710 - val_recall: 0.8326
Epoch 4/100
222/222 [==============================] - 2s 9ms/step - loss: 0.2655 - recall: 0.8509 - val_loss: 0.2381 - val_recall: 0.8750
Epoch 5/100
222/222 [==============================] - 2s 10ms/step - loss: 0.2483 - recall: 0.8603 - val_loss: 0.2410 - val_recall: 0.8427
Epoch 6/100
222/222 [==============================] - 2s 9ms/step - loss: 0.2385 - recall: 0.8654 - val_loss: 0.2286 - val_recall: 0.8965
Epoch 7/100
222/222 [==============================] - 2s 11ms/step - loss: 0.2312 - recall: 0.8705 - val_loss: 0.2218 - val_recall: 0.9082
Epoch 8/100
222/222 [===

In [9]:
from sklearn.metrics import precision_score, recall_score

y_pred = model.predict(X_test) > 0.5
(
    precision_score(y_test, y_pred),
    recall_score(y_test, y_pred)
)

739/739 [==============================] - 2s 2ms/step


(0.9682656053623796, 0.9356340451371319)

In [11]:
#model.save("rnn_model.h5")